# Look at the big picture.


## Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from datetime import datetime
from statistics import median
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%load_ext pycodestyle_magic

In [ ]:
# Activamos las alertas de estilo
%pycodestyle_on

## Performance Metric

In [2]:
def performance(y_true: list, y_pred: list) -> float:
    """

    """

    CF = [1 for _ in y_true if y_pred > y_true] / len(y_pred) * 100
    RMSE = mean_squared_errors(y_true, y_pred)
    rRMSE = RMSE / median(y_true)
    metric = (0.7 * rRMSE) + (0.3 * (1 - CF))

    return metric

# Get the data

In [3]:
dtypes = {
    "fecha": "str",
    "id": "category",
    "visitas": "Int64",
    "categoria_uno": "category",
    "categoria_dos": "category",
    "estado": "category",
    "precio": "Float64",
    "dia_atipico": "category",
    "campaña": "category",
    "antiguedad": "Int64",
    "unidades_vendidas": "Int64"    
}

# Read the data
data = pd.read_csv('./data/Modelar_UH2021.txt', delimiter="|", dtype=dtypes, decimal=',')

## Overview the data

In [4]:
data.head()

,fecha,id,visitas,categoria_uno,categoria_dos,estado,precio,dia_atipico,campaña,antiguedad,unidades_vendidas
0,1/6/2015 0:00:00,21972,0,C,75,No Rotura,NaN,0,0,5241,0
1,1/6/2015 0:00:00,23910,5,C,170,No Rotura,6.07,0,0,5241,3
2,1/6/2015 0:00:00,24306,13,A,46,No Rotura,NaN,0,0,NaN,0
3,1/6/2015 0:00:00,24306,13,A,46,No Rotura,NaN,0,0,NaN,0
4,1/6/2015 0:00:00,27144,15,E,230,No Rotura,NaN,0,0,4064,0


In [5]:
data.describe()

,visitas,precio,antiguedad,unidades_vendidas
count,4.045022e+06,1.402111e+06,3.170857e+06,4.045022e+06
mean,1.721371e+02,3.424319e+01,1.011114e+03,4.693434e+00
std,6.889116e+02,2.330943e+01,7.162509e+02,2.237403e+01
min,0.000000e+00,3.570000e+00,1.260000e+02,0.000000e+00
25%,7.000000e+00,1.652000e+01,5.240000e+02,0.000000e+00
50%,3.500000e+01,2.689000e+01,7.950000e+02,0.000000e+00
75%,1.300000e+02,4.535000e+01,1.244000e+03,3.000000e+00
max,1.200450e+05,1.757800e+02,5.310000e+03,4.881000e+03


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4045022 entries, 0 to 4045021
Data columns (total 11 columns):
fecha                object
id                   category
visitas              Int64
categoria_uno        category
categoria_dos        category
estado               category
precio               float64
dia_atipico          category
campaña              category
antiguedad           Int64
unidades_vendidas    Int64
dtypes: Int64(3), category(6), float64(1), object(1)
memory usage: 196.9+ MB


# Discover and visualize the data to gain insights

# Prepare the data for Machine Learning algorithms

## Preprocessing functions 

### Time format

In [7]:
class time_format(BaseEstimator, TransformerMixin):

    def __init__(self, document='Modelar'):
        self.document = document

    def fit(self, X, y=None):
        # nothing else to do
        return self


    def transform(self, data):
        """
        Take the dataframe and trate the 'fecha' attribute depending of the document.
        If it is the 'Modelar' document, the variable will be equal to the content
        before the first space.
        If it it the 'Estimar' document, the variable will be transformed to the
        original format

        Original Format: 'DD/MM/AAAA' in (text format)


        """

        temp = ''

        if self.document == 'Modelar':
            data['fecha'] = pd.to_datetime(data['fecha'],infer_datetime_format=True).apply(lambda x : x.strftime('%d/%m/%Y'))
        elif self.document == 'Estimar':
            # agregar la modificación para el documento Estimar
            data['fecha']
        else:
            print('Unknown document!!!')


        return data

### Drop duplicates

In [8]:
class drop_dup(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        # nothing else to do
        return self


    def transform(self, data):
        """
        Take the dataframe and return the dataframe without duplicates.

        """

        return data.drop_duplicates()

### Data Preprocessing Pipeline

In [9]:
data_prep_pipeline = Pipeline([
         ('drop_dup', drop_dup()),
         ('time_format', time_format(document='Modelar'))
])


data_prepared = data_prep_pipeline.fit_transform(data)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Missing Values

In [10]:
## Get the missing values per attribute
data.isnull().sum()

fecha                      0
id                         0
visitas                    0
categoria_uno              0
categoria_dos           5844
estado                     0
precio               2642911
dia_atipico                0
campaña                    0
antiguedad            874165
unidades_vendidas          0
dtype: int64

In [11]:
data_final = data_prepared.copy()
data_final['fecha'] = data_final['fecha'].apply( lambda i : datetime.strptime(i , "%d/%m/%Y"))  
data_final.sort_values(by=['fecha'], inplace=True, ascending=True)
data_final = data_final.reset_index(drop=True)

In [12]:
data_final[data_final['id'] == str(23910)]

,fecha,id,visitas,categoria_uno,categoria_dos,estado,precio,dia_atipico,campaña,antiguedad,unidades_vendidas
2770,2015-01-06,23910,5,C,170,No Rotura,6.07,0,0,5241,3
8340,2015-01-07,23910,0,C,170,No Rotura,NaN,0,0,5241,0
11114,2015-01-08,23910,2,C,170,No Rotura,NaN,0,0,5241,0
13907,2015-01-09,23910,11,C,170,No Rotura,6.12,0,0,5241,21
19458,2015-01-10,23910,12,C,170,No Rotura,NaN,0,0,5241,0
23624,2015-01-11,23910,3,C,170,No Rotura,NaN,-1,0,5241,0
27796,2015-01-12,23910,6,C,170,No Rotura,NaN,0,0,5241,0
30589,2015-02-06,23910,3,C,170,No Rotura,NaN,0,0,5241,0
36136,2015-02-07,23910,6,C,170,No Rotura,6.08,1,0,5241,3
40310,2015-02-08,23910,5,C,170,No Rotura,NaN,0,0,5241,0


In [13]:
last_values = dict([ (i, 0) for i in data_final.id.unique() ])

df_index = 0

for index, value in zip(data_final.id, data_final.precio):
    if pd.notna(float(value)):
        last_values[index] = value
    else:
        data_final.at[df_index, 'precio'] = last_values[index]
   
    df_index += 1

In [14]:
data_final[data_final['id'] == str(23910)]

,fecha,id,visitas,categoria_uno,categoria_dos,estado,precio,dia_atipico,campaña,antiguedad,unidades_vendidas
2770,2015-01-06,23910,5,C,170,No Rotura,6.07,0,0,5241,3
8340,2015-01-07,23910,0,C,170,No Rotura,6.07,0,0,5241,0
11114,2015-01-08,23910,2,C,170,No Rotura,6.07,0,0,5241,0
13907,2015-01-09,23910,11,C,170,No Rotura,6.12,0,0,5241,21
19458,2015-01-10,23910,12,C,170,No Rotura,6.12,0,0,5241,0
23624,2015-01-11,23910,3,C,170,No Rotura,6.12,-1,0,5241,0
27796,2015-01-12,23910,6,C,170,No Rotura,6.12,0,0,5241,0
30589,2015-02-06,23910,3,C,170,No Rotura,6.12,0,0,5241,0
36136,2015-02-07,23910,6,C,170,No Rotura,6.08,1,0,5241,3
40310,2015-02-08,23910,5,C,170,No Rotura,6.08,0,0,5241,0


Imputar la variable antiguedad por la media y la categoría dos por 0(de Momento).

In [15]:
# Get the median value
median = data_final['antiguedad'].median()

# Fill the NA Values with the median
data_final['antiguedad'].fillna(median, inplace=True)
#data_final['categoria_dos'].fillna(, inplace=True) # De momento dejadlo así, ya estoy buscando la manera de hacerlo con clustering


In [16]:
#last_values = dict([ (i, 0) for i in data_final.id.unique() ])
list_values = []
df_index = 0
null_count = 0

for index, value in zip(data_final.id, data_final.categoria_dos):
    if  pd.notna(float(value)):
        # last_values[index] = value
        list_values.append(value)
    else:
        null_count += 1
        list_values.append(list_values[len(list_values)-1])
   
    df_index += 1
    
print(len(list_values), null_count)
data_final['categoria_dos'] = np.array(list_values)
data_final['categoria_dos'] = data_final['categoria_dos'].astype('category')

2040037 4393


In [17]:
data_final['dia_atipico'] = data_final['dia_atipico'].replace(['0', '1', '-1'], ["Venta_Normal", "Venta_alta", "Venta_Baja"])

In [18]:
data_final.dia_atipico

0          Venta_Normal
1          Venta_Normal
2          Venta_Normal
3          Venta_Normal
4          Venta_Normal
5          Venta_Normal
6          Venta_Normal
7          Venta_Normal
8          Venta_Normal
9          Venta_Normal
10         Venta_Normal
11         Venta_Normal
12         Venta_Normal
13         Venta_Normal
14         Venta_Normal
15         Venta_Normal
16         Venta_Normal
17         Venta_Normal
18         Venta_Normal
19         Venta_Normal
20         Venta_Normal
21         Venta_Normal
22         Venta_Normal
23         Venta_Normal
24         Venta_Normal
25         Venta_Normal
26         Venta_Normal
27         Venta_Normal
28         Venta_Normal
29         Venta_Normal
               ...     
2040007    Venta_Normal
2040008    Venta_Normal
2040009    Venta_Normal
2040010    Venta_Normal
2040011    Venta_Normal
2040012    Venta_Normal
2040013    Venta_Normal
2040014    Venta_Normal
2040015    Venta_Normal
2040016    Venta_Normal
2040017    Venta

### One Hot Encoding

In [19]:
data_final.isnull().sum()

fecha                0
id                   0
visitas              0
categoria_uno        0
categoria_dos        0
estado               0
precio               0
dia_atipico          0
campaña              0
antiguedad           0
unidades_vendidas    0
dtype: int64

In [20]:
# concatenamos las dos columnas 
#data_final['categoria'] = data_final["categoria_uno"].str.cat(data_final.categoria_dos, sep ="") 
#data_final['categoria']
#data_final.drop(columns=['categoria_uno', 'categoria_dos'], inplace = True)
#data_final

In [23]:
encoder = OneHotEncoder()

enc_df = pd.DataFrame(encoder.fit_transform(data_final[['estado','dia_atipico','categoria_uno','categoria_dos']]).toarray())

data_model = data_final.join(enc_df)

data_model = data_model.drop(columns=['estado','dia_atipico','categoria_uno','categoria_dos'])

data_model

,fecha,id,visitas,precio,campaña,antiguedad,unidades_vendidas,0,1,2,...,191,192,193,194,195,196,197,198,199,200
0,2015-01-06,21972,0,0.00,0,5241,0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-01-06,327312,12,0.00,0,580,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-01-06,327330,7,0.00,0,580,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-01-06,327348,16,0.00,0,580,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-06,327380,2,0.00,0,580,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2015-01-06,327386,3,0.00,0,580,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2015-01-06,327468,27,0.00,0,580,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2015-01-06,327474,20,0.00,0,580,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2015-01-06,327480,19,0.00,0,580,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2015-01-06,327518,11,0.00,0,831,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Outliers

No debe haber outliers


# Select a model and train it

# Fine-tune your model

# Present your solution